In [1]:
import os
import numpy as np
import seaborn as sns
import wandb
#from wandb.keras import WandbCallback
from keras import callbacks
import keras
import DeepSetNeuralNetArchitecture as DSNNA
import ParticleTransformer as ParT
from hffragDeepSetsMultivariate import PredictOnEpoch
from DeepSetNeuralNetArchitecture import LogNormal_Loss_Function
from DeepSetNeuralNetArchitecture import Mean_Squared_Error
from hffragDeepSetsMultivariate import hffragDeepSets
from HffragDeepSetsProjection import DeepSetsProjection
from hffragDeepSetsMultivariate import Multivariate_Gaussian_Negative_Likelihood_Loss_Curve
import keras.backend as k
import uproot
import awkward as ak
import sklearn as sk
from numpy.lib.recfunctions import structured_to_unstructured
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
import pandas as pd
from hffrag import fixedbinning
from hffrag import binneddensity
import numpy_indexed as npi
from keras.utils.vis_utils import plot_model
from timeit import default_timer as timer
import matplotlib.pyplot as plt
%matplotlib inline

KeyboardInterrupt: 

In [2]:
plt.style.use("default")
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
plt.rc('text',usetex = False)
plt.rc('font',family = 'Times New Roman')

In [3]:
# The data is being stored in a tree datastructure.
# We access the charm root using this command
tree = uproot.open("hffrag.root:CharmAnalysis")

In [4]:
# Initial parameters
MASKVAL = -999 # This value is introduced to ensure arrays are regular (Of the same size). They will be masked later by the network
MAXTRACKS = 32 # This value is the maximum number of tracks allowed per event
BATCHSIZE = 64 # This is the batch size of the mini batches used during training
EPOCHS = 1000 # This is the default number of epochs for which the neural network will train providing that early stopping does not occur
MAXEVENTS = 1e20 #This is the maximum number of events that will the program will accept
LR = 1e-4 #This is the default learning rate

In [5]:
# Select the features we wish to study
track_features = ["AnalysisTracks_pt", "AnalysisTracks_eta", "AnalysisTracks_phi", "AnalysisTracks_z0sinTheta",
                  "AnalysisTracks_d0sig", "AnalysisTracks_d0", "AnalysisTracks_d0sigPV", "AnalysisTracks_d0PV"]
jet_features = ["AnalysisAntiKt4TruthJets_pt", "AnalysisAntiKt4TruthJets_eta", "AnalysisAntiKt4TruthJets_phi",
                "AnalysisAntiKt4TruthJets_ghostB_pt", "AnalysisAntiKt4TruthJets_ghostB_eta","AnalysisAntiKt4TruthJets_ghostB_phi"]

In [6]:
# Read in the dat from the root file
features = tree.arrays(jet_features+track_features, entry_stop=MAXEVENTS)

In [7]:
# Select the events of interest
events = features[ak.sum(
    features["AnalysisAntiKt4TruthJets_pt"] > 25000, axis=1) > 0]

In [8]:
# Displays the number of jets being trained on
jets = events[jet_features][:, 0]
print("The number of jets to train on is: ", len(jets))
print("The number of track features is: ",len(track_features))

The number of jets to train on is:  141329
The number of track features is:  8


In [9]:
# Select tracks from the events
tracks = events[track_features]

# Match the tracks to the jets
matchedtracks = tracks[DSNNA.Match_Tracks(jets, tracks)]

# Pad and Flatten the data
matchedtracks = DSNNA.flatten(matchedtracks, MAXTRACKS)

In [10]:
# Identify the the bottom jets and their associated tracks
bjets = ak.sum(jets["AnalysisAntiKt4TruthJets_ghostB_pt"] > 5000, axis=1) > 0
jets = jets[bjets]

# Obtain the pt, eta and phi of each b hadron jet
bhads_pt = jets["AnalysisAntiKt4TruthJets_ghostB_pt"][:, 0].to_numpy()
bhads_eta = jets["AnalysisAntiKt4TruthJets_ghostB_eta"][:,0].to_numpy()
bhads_phi = jets["AnalysisAntiKt4TruthJets_ghostB_phi"][:,0].to_numpy()

jets_pt = jets["AnalysisAntiKt4TruthJets_pt"].to_numpy()
jets_eta = jets["AnalysisAntiKt4TruthJets_eta"].to_numpy()
jets_phi = jets["AnalysisAntiKt4TruthJets_phi"].to_numpy()
b_jets = np.stack([jets_pt,jets_eta,jets_phi], axis = -1)

bhads = np.stack([bhads_pt,bhads_eta,bhads_phi],axis = -1) #Combine the momentum, eta and phi for each jet into one array

print("There are {} outputs".format(np.shape(bhads)[1])) # Display the number of target features the neural network will predict
matchedtracks = matchedtracks[bjets]
print("There are {} inputs".format(np.shape(matchedtracks)[1])) # Display the number of target features the neural network will use in it's ppredictions

There are 3 outputs
There are 32 inputs


In [11]:
# Transform the jet and tracks to unstructed data.
jets = structured_to_unstructured(jets[jet_features[:-3]])
matchedtracks = structured_to_unstructured(matchedtracks)

In [12]:
# Convert the coordinates of the b jets and tracks to cartesian coordinates
tracks_p = DSNNA.pt_eta_phi_2_px_py_pz_tracks(matchedtracks.to_numpy())
bhads = DSNNA.pt_eta_phi_2_px_py_pz_jets(bhads)
b_jets = DSNNA.pt_eta_phi_2_px_py_pz_jets(b_jets)

#Combine the momenta of the tracks with the rest of the track features to form the track dataset
tracks = np.concatenate([tracks_p,matchedtracks[:,:,3:].to_numpy()],axis = 2)

/home/physics/phujdj/DeepLearningParticlePhysics/DeepSetNeuralNetArchitecture.py:104: RuntimeWarning: overflow encountered in sinh
  pzs = np.where(mask1 | mask3, pts, pts * np.sinh(etas))


In [13]:
b_jets_mag = np.linalg.norm(b_jets, axis = 1)

bhads_fractions_px = bhads[:,0]/b_jets[:,0]
bhads_fractions_py = bhads[:,1]/b_jets[:,1]
bhads_fractions_pz = bhads[:,2]/b_jets[:,2]
print(bhads_fractions_px.shape)

bhads_fractions = np.stack([bhads_fractions_px,bhads_fractions_py, bhads_fractions_pz], axis = -1)
print(bhads_fractions.shape)

print(bhads_fractions[0])
print(bhads[0,0]/b_jets[0,0])
print(np.min(bhads_fractions),np.max(bhads_fractions))

print(bhads_fractions_px[0])
print(bhads[0,0]/b_jets[0,0])
print(np.min(bhads_fractions_px),np.max(bhads_fractions_px))

sum_px_tracks = np.sum(tracks[:,:,0], axis = 1)
sum_py_tracks = np.sum(tracks[:,:,1], axis = 1)
sum_pz_tracks = np.sum(tracks[:,:,2], axis = 1)

bhads_projection = ((bhads*b_jets).sum(axis = 1))/(b_jets_mag**2)

b_jets = np.stack([b_jets[:,0], b_jets[:,1], b_jets[:,2], sum_px_tracks, sum_py_tracks, sum_pz_tracks], axis = -1)
bhads_targets = np.stack([bhads_fractions_px,bhads_fractions_py, bhads_fractions_pz, bhads_projection], axis = -1)


(68143,)
(68143, 3)
[0.92818976 0.92693198 0.92622171]
0.928189759678061
-1196.4922484782348 373.05142412393263
0.928189759678061
0.928189759678061
-230.22990301009094 171.10816219196536


In [14]:
tracks_fractions_px = tracks[:,:,0]/b_jets[:,0,np.newaxis]
tracks_fractions_py = tracks[:,:,1]/b_jets[:,1,np.newaxis]
tracks_fractions_pz = tracks[:,:,2]/b_jets[:,2,np.newaxis]

Track_fractions = np.stack([tracks_fractions_px,tracks_fractions_py, tracks_fractions_pz], axis = -1)
print(Track_fractions.shape)

print(Track_fractions.shape)
print(tracks[0,0,0]/b_jets[0,0])
print(np.mean(Track_fractions),np.std(Track_fractions))

Tracks_projection = ((tracks[:,:,:3]*b_jets[:,np.newaxis,:3]).sum(axis = 2)/(b_jets_mag[:,np.newaxis]**2))
print(Tracks_projection.shape)

(68143, 32, 3)
(68143, 32, 3)
0.11796763971634215
0.032278004649905545 6.120054830548441
(68143, 32)


In [15]:
Scaler_tracks = StandardScaler()
Num_events,Num_tracks,Num_features = np.shape(tracks)
Scaled_tracks = np.reshape(tracks, newshape=(-1,Num_features))
tracks_scaled = Scaler_tracks.fit_transform(Scaled_tracks)
tracks_scaled = np.reshape(tracks_scaled, newshape= (Num_events,Num_tracks,Num_features))
print(np.shape(tracks_scaled))
print(tracks_scaled[0,0,:])

Scaler_jets = StandardScaler()
Num_events,Num_features = np.shape(b_jets)
b_jets_scaled = np.reshape(b_jets, newshape=(-1,Num_features))
b_jets_scaled = Scaler_jets.fit_transform(b_jets)
b_jets_scaled = np.reshape(b_jets_scaled, newshape= (Num_events,Num_features))
print(np.shape(b_jets_scaled))
print(b_jets_scaled[0,:])

(68143, 32, 8)
[0.26189855 0.38175348 0.10230412 1.42959932 1.58783932 1.42992229
 1.60955267 1.42990682]
(68143, 6)
[0.60802015 1.59589996 0.53448783 0.11311969 0.27945027 0.08234729]


In [16]:
Tracks_input = np.concatenate([tracks_scaled, Track_fractions], axis = -1)
print(Tracks_input.shape)

(68143, 32, 11)


In [17]:
b_jets_input = np.concatenate([b_jets_scaled, Tracks_projection], axis = -1)
print(b_jets_input.shape)

(68143, 38)


In [18]:
# Split the data into training and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(
    Tracks_input, bhads_targets, train_size=0.7, random_state = 42)

In [19]:
# Split the data into training and validation sets.
X_train_b_jets, X_valid_b_jets, y_train_b_jets, y_valid_b_jets = train_test_split(
    b_jets_input, bhads_targets, train_size=0.7, random_state = 42)

In [20]:
X_train_event, y_train_event = np.array([X_train[0]]), np.array([y_train[0]])
X_valid_event, y_valid_event = np.array([X_valid[0]]), np.array([y_valid[0]])
print(np.shape(X_train),np.shape(y_train))
print(np.shape(X_train_event),np.shape(y_train_event))

(47700, 32, 11) (47700, 4)
(1, 32, 11) (1, 4)


In [21]:
#Check for the of the training and validation sets
print(np.shape(X_train), np.shape(X_valid))
print(np.shape(X_train_b_jets), np.shape(X_valid_b_jets))
print(np.shape(y_train), np.shape(y_valid))

(47700, 32, 11) (20443, 32, 11)
(47700, 38) (20443, 38)
(47700, 4) (20443, 4)


In [22]:
np.shape(Tracks_input)[2]

11

In [23]:
#Cyclical Learning Rate Scheduler:
steps_per_epoch = len(X_train)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate = 1e-4,
maximal_learning_rate = 0.01,
scale_fn = lambda x: 1/(2**(x-1)),
step_size = 2.0 * steps_per_epoch
)
class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs = {}):
        self.logs = []
    def on_epoch_begin(self, epoch, logs ={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs = {}):
        self.logs.append(timer() - self.starttime)
        
# Builds the deep neural network
track_layers = [100 for x in range(3)]
jet_layers = [100 for x in range(3)]
b_jets_layers = [100 for x in range(3)]

len1 = [len(track_features)]+track_layers
print(len1)

#Initializers the optimizer used for training the network
optimizer = tf.keras.optimizers.Nadam(LR)

#Build a DeepSets Projection Neural Network
DeepSetProjector = DeepSetsProjection([np.shape(Tracks_input)[2]]+track_layers, jet_layers, b_jets_layers, np.shape(y_train)[1], 0.0001, np.shape(b_jets_input)[1],optimizer, loss_curve=tf.keras.losses.MSE)

[8, 100, 100, 100, 100, 100, 100]


2023-02-04 20:29:51.671539: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-04 20:29:51.671571: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-04 20:29:51.671595: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vonneumann.csc.warwick.ac.uk): /proc/driver/nvidia/version does not exist
2023-02-04 20:29:51.673235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
#Summarises the DeepSetsProjector Set Neural Network Architecture
DeepSetProjector.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 38)]         0           []                               
                                                                                                  
 dense_19 (Dense)               (None, 100)          3900        ['input_4[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, None, 11)]   0           []                               
                                                                                                  
 dropout_6 (Dropout)            (None, 100)          0           ['dense_19[1][0]']               
                                                                                              

In [25]:
# Introduce early_stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    min_delta=0.00001,  # The minimum amount of change to count as an improvement
    patience=20,  # The number of epochs to wait before stopping
    restore_best_weights=True,  # Keep the best weights
)
# Prevent spikes in the validation and training loss due to the gradient descent kicking the network out of a local minima
reduce_learn_on_plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.80, patience=5, min_lr=1e-9)

# Save the weights of the model to allow reuse in future.
path = "/home/physics/phujdj/DeepLearningParticlePhysics/CheckPointsDeepNet/DeepNetWeights&Biases.ckpt"
checkpoint_dir = os.path.dirname(path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=path,
                                                 save_weights_only=True, verbose=0, save_freq = 100*BATCHSIZE)
#Timer
cb = TimingCallback()

#Weight&Biases Callback:
#Wanda = WandbCallback(save_graph = True,save_weights_only = True, log_weights = True, log_gradients = True, log_evaluation = True, training_data = (X_train,y_train), validation_data = (X_valid,y_valid), log_batch_frequency = 5)

# Learning Scheduler:
exponential_decay_fn = DSNNA.expontial_decay(lr0 = LR,s = 30)
learning_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)


In [26]:
X_train.shape

(47700, 32, 11)

In [27]:
history  = DeepSetProjector.fit(
    (X_train, X_train_b_jets), y_train,
    validation_data = ((X_valid,X_valid_b_jets), y_valid),
    epochs = 1000,
    batch_size = BATCHSIZE,
    callbacks = [early_stopping, cp_callback, cb, reduce_learn_on_plateau],
    )

Epoch 1/1000


2023-02-04 20:30:08.397635: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f23040e6aa0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-04 20:30:08.397677: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2023-02-04 20:30:09.100298: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-04 20:30:09.114420: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. model/dropout_6/dropout/random_uniform/RandomUniform


  3/746 [..............................] - ETA: 23s - loss: 12.6705    

2023-02-04 20:30:32.243843: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


746/746 [==============================] - 92s 71ms/step - loss: 17.9475 - val_loss: 7.7062 - lr: 1.0000e-04
Epoch 2/1000
746/746 [==============================] - 25s 33ms/step - loss: 14.1376 - val_loss: 5.0285 - lr: 1.0000e-04
Epoch 3/1000
321/746 [===========>..................] - ETA: 12s - loss: 4.2464

KeyboardInterrupt: 

In [ ]:
# Plot the loss and validation curves vs epoch
history_df = pd.DataFrame(history.history)
np.log(history_df.loc[:, ["loss","val_loss"]]).plot()
history_df.to_csv('/home/physics/phujdj/DeepLearningParticlePhysics/history.csv')

In [ ]:
# Output to the console the minimum epoch
print("Minimum validation loss: {}".format(history_df["loss"].min()))

In [ ]:
#Evaluate the entire performance of the model
loss = DeepSetProjector.evaluate((Tracks_input,b_jets_input),bhads_targets,verbose = 2)
print("The Loaded DeepNet has loss: ", loss)

In [ ]:
Predictions = DeepSetProjector.predict((Tracks_input, b_jets_input))
Predictions_px = Predictions[:,0] * b_jets[:,0]
Predictions_py = Predictions[:,1] * b_jets[:,1]
Predictions_pz = Predictions[:,2] * b_jets[:,2]
Predictions_Projected = Predictions[:,3]
print(Predictions.shape)

In [ ]:
Error_px = bhads[:,0] - Predictions_px
Pull_bhads_px = Error_px/np.std(bhads[:,0])

In [ ]:
Error_py = bhads[:,1] - Predictions_py
Pull_bhads_py = Error_py/np.std(bhads[:,1])

In [ ]:
Error_pz = bhads[:,2] - Predictions_pz
Pull_bhads_pz = Error_pz/np.std(bhads[:,2])

In [ ]:
Error_projection = bhads_targets[:,-1] - Predictions_Projected
Pull_bhads_pz = Error_pz/np.std(bhads_targets[:,-1])

In [ ]:
print(np.mean(Error_px))
print(np.std(Error_px))
binneddensity(Error_px, fixedbinning(-40000,40000,100),xlabel = "Error_px")

In [ ]:
print(np.mean(Error_py))
print(np.std(Error_py))
binneddensity(Error_py, fixedbinning(-40000,40000,100),xlabel = "Error_py")

In [ ]:
print(np.mean(Error_pz))
print(np.std(Error_pz))
binneddensity(Error_pz, fixedbinning(-40000,40000,100),xlabel = "Error_pz")

In [ ]:
print(np.mean(Error_projection))
print(np.std(Error_projection))
binneddensity(Error_projection, fixedbinning(-1,1,100),xlabel = "Error_projection")

In [ ]:
print(np.mean(Pull_bhads_px))
print(np.std(Pull_bhads_px))
binneddensity(Pull_bhads_px, fixedbinning(-1,1,100),xlabel = "Pull_px")

In [ ]:
print(np.mean(Pull_bhads_py))
print(np.std(Pull_bhads_py))
binneddensity(Pull_bhads_py, fixedbinning(-1,1,100),xlabel = "Pull_py")

In [ ]:
print(np.mean(Pull_bhads_pz))
print(np.std(Pull_bhads_pz))
binneddensity(Pull_bhads_pz, fixedbinning(-100000,100000,100),xlabel = "Pull_pz")

In [ ]:
fig,ax = plt.subplots(figsize = (12,12))
sns.scatterplot(
    y = Predictions_px,
    x = bhads[:,0],
    color = "blue"
)
ax.set_xlim([-400000,400000])
ax.set_ylim([-400000,400000])
ax.set_title("Scatterplot of the true vs pred X momenta")
ax.set_xlabel("The true X momenta of the tracks from each event")
ax.set_ylabel("The predicted X momenta of b hadron jets")

In [ ]:
fig,ax = plt.subplots(figsize = (12,12))
sns.scatterplot(
    y = Predictions_py,
    x = bhads[:,1],
    color = "green"
)
ax.set_xlim([-400000,400000])
ax.set_ylim([-400000,400000])
ax.set_title("Scatterplot of the true vs pred Y momenta")
ax.set_xlabel("The true Y momenta of the tracks from each event")
ax.set_ylabel("The predicted Y momenta of b hadron jets")

In [ ]:
fig,ax = plt.subplots(figsize = (12,12))
sns.scatterplot(
    y = Predictions_pz,
    x = bhads[:,2],
    color = "orange"
)
ax.set_title("Scatterplot of the true vs pred Z momenta")
ax.set_xlabel("The true Z momenta of the tracks from each event")
ax.set_ylabel("The predicted Z momenta of b hadron jets")

In [ ]:
Predictions_cart = np.stack([Predictions_px, Predictions_py, Predictions_pz], axis = -1)
Predictions_Projection = ((Predictions_cart*b_jets[:,:3]).sum(axis = 1))/(b_jets_mag**2)

In [ ]:
fig,ax = plt.subplots(figsize = (12,12))
sns.scatterplot(
    y = Predictions_Projection,
    x = bhads_projection,
    color = "orange"
)
ax.set_title("Scatterplot of the true vs pred Z momenta")
ax.set_xlabel("The true Z momenta of the tracks from each event")
ax.set_ylabel("The predicted Z momenta of b hadron jets")

In [ ]:
print(np.mean(Predictions_Projection))
print(np.std(Predictions_Projection))
binneddensity(Predictions_Projection, fixedbinning(0,2,100),xlabel = "Plot")